In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

/home/ada/humor/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
transcripts = pd.read_csv('/home/ada/humor/data/stand_up_dataset/standup_transcripts.csv')

2 billion

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.45it/s]


In [4]:
results = []
for index, row in transcripts.iterrows():
    comedian = row['comedian']
    transcript = row['transcript'] 
    
    instruction = f"Extract the key humorous lines and punchlines for this stand-up comedy transcript. Focus on the quotes highlighting the main comedic moments.{transcript} List of quotes:"
    input_ids = tokenizer(instruction, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(input_ids=input_ids, max_new_tokens=120)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = generated_text.replace(instruction, "").strip()
    
    results.append({'comedian': comedian, 'sentence': response})

In [5]:
results

[{'comedian': 'Anthony_Jeselnik',
  'sentence': '"When I was a kid, I used to fantasize about getting older, growing up and having money, and buying my mom nice things for her birthday."\n"When I was a kid, we were poor. So poor I remember, just so I could go to my senior prom, just so I could go to my senior prom, I had to sell my U.S. passport on the street. Sold my passport on the street for 300 bucks to get to go to my prom. Of course this was before 9/11 so… my bad, everybody'},
 {'comedian': 'Anthony_Jeselnik_2',
  'sentence': '“I’ve never talked to a group of people without getting paid a lot of money.”\n“Anthony, just go up there and tell a story. Find one moment about you and your grandma you can share with everybody. And don’t tell a joke. Try not to.”\n“You know what my favorite memory was? When I was like four years old, before I learned to read, Grandma would curl up with me on the couch, she had this Southern accent, and she would read to me. She would read Mark Twain to 